# Hybrid quantum-classical feed forward neural network for solving the TSP.

In [ ]:
!pip install -q cirq==0.13.1


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
markdown 3.3.7 requires importlib-metadata>=4.4; python_version < "3.10", but you have importlib-metadata 3.10.1 which is incompatible.


In [ ]:
!pip install -q tensorflow==2.7.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyquil 3.0.1 requires importlib-metadata<4.0.0,>=3.7.3; python_version < "3.8", but you have importlib-metadata 4.12.0 which is incompatible.


In [ ]:
!pip install -q tensorflow_quantum==0.6.1


In [ ]:
!pip install -q numpy==1.22.3


ERROR: Could not find a version that satisfies the requirement numpy==1.22.3 (from versions: 1.3.0, 1.4.1, 1.5.0, 1.5.1, 1.6.0, 1.6.1, 1.6.2, 1.7.0, 1.7.1, 1.7.2, 1.8.0, 1.8.1, 1.8.2, 1.9.0, 1.9.1, 1.9.2, 1.9.3, 1.10.0.post2, 1.10.1, 1.10.2, 1.10.4, 1.11.0, 1.11.1, 1.11.2, 1.11.3, 1.12.0, 1.12.1, 1.13.0rc1, 1.13.0rc2, 1.13.0, 1.13.1, 1.13.3, 1.14.0rc1, 1.14.0, 1.14.1, 1.14.2, 1.14.3, 1.14.4, 1.14.5, 1.14.6, 1.15.0rc1, 1.15.0rc2, 1.15.0, 1.15.1, 1.15.2, 1.15.3, 1.15.4, 1.16.0rc1, 1.16.0rc2, 1.16.0, 1.16.1, 1.16.2, 1.16.3, 1.16.4, 1.16.5, 1.16.6, 1.17.0rc1, 1.17.0rc2, 1.17.0, 1.17.1, 1.17.2, 1.17.3, 1.17.4, 1.17.5, 1.18.0rc1, 1.18.0, 1.18.1, 1.18.2, 1.18.3, 1.18.4, 1.18.5, 1.19.0rc1, 1.19.0rc2, 1.19.0, 1.19.1, 1.19.2, 1.19.3, 1.19.4, 1.19.5, 1.20.0rc1, 1.20.0rc2, 1.20.0, 1.20.1, 1.20.2, 1.20.3, 1.21.0rc1, 1.21.0rc2, 1.21.0, 1.21.1, 1.21.2, 1.21.3, 1.21.4, 1.21.5, 1.21.6)
ERROR: No matching distribution found for numpy==1.22.3


In [ ]:
!pip install -q sympy==1.8


In [ ]:
!pip install -q matplotlib==3.5.1

In [ ]:
import cirq
import tensorflow as tf
import tensorflow_quantum as tfq
import numpy as np
import sympy
import matplotlib.pyplot as plt
import itertools

%matplotlib inline

## TSP 


In [ ]:
class TSP:
    def __init__(self, number_of_cities, coords_range=(0, 10000)):
        self.number_of_cities = number_of_cities
        self.coords_range = coords_range
        self.cities_coords = self.get_cities()
        self.distance_matrix = self.calculate_distance_matrix()
        self.normalized_distance_matrix = self.normalize_distance_matrix()
    
    def get_cities(self):
        cities_coords = np.random.randint(self.coords_range[0], self.coords_range[1], size = (self.number_of_cities, 2))
        return cities_coords
           
    def normalize_cities(self):
        max_coords = np.amax(self.cities_coords, axis=0)
        normalized_cities_coords = np.divide(self.cities_coords, max_coords)
        return normalized_cities_coords

    def calculate_distance_between_points(self, point_A, point_B):
        return np.sqrt((point_A[0] - point_B[0])**2 + (point_A[1] - point_B[1])**2)
    
    def calculate_distance_matrix(self):
        distance_matrix = np.zeros((self.number_of_cities, self.number_of_cities))
        for i in range(self.number_of_cities):
            for j in range(i, self.number_of_cities):
                distance_matrix[i][j] = self.calculate_distance_between_points(self.cities_coords[i], self.cities_coords[j])
                distance_matrix[j][i] = distance_matrix[i][j]
        return distance_matrix 
    
    def normalize_distance_matrix(self):
        return np.divide(self.distance_matrix, np.max(self.distance_matrix))

##  QAOA

In [ ]:
class QAOA_TSP:
    def __init__(self, tsp_instance, p=1, A_1=4, A_2=4, B=1):
        self.tsp_instance = tsp_instance
        self.qubits = cirq.GridQubit.rect(1, tsp_instance.number_of_cities**2)
        self.p = p
        self.weights = {'cost_weight': B, 
                        'constraint_each_visited': A_1, 
                        'constraint_each_visited_once': A_2}
        self.parameters = self.generate_sympy_parameters()
        self.cost_operator = self.create_cost_operator()
        self.circuit = self.create_qaoa_circuit()
    
    def calc_bit(self, i, t):
        return i + t * self.tsp_instance.number_of_cities
    
    def x(self, i, t):
        x = self.calc_bit(i, t)
        qubit = self.qubits[x]
        return cirq.PauliString(0.5, cirq.I(qubit)) - cirq.PauliString(0.5, cirq.Z(qubit))
        
    def create_hadamard_circuit_layer(self):
        hadamard_circuit_layer = cirq.Circuit()
        for qubit in self.qubits:
            hadamard_circuit_layer += cirq.H(qubit)
        return hadamard_circuit_layer
    
    def generate_sympy_parameters(self):
        return sympy.symbols('parameter_:%d'%(2*self.p))

    def create_cost_operator(self):
        A_1 = self.weights['constraint_each_visited']
        A_2 = self.weights['constraint_each_visited_once']
        B = self.weights['cost_weight']
        
        cost_of_constraint_each_visited = 0    
        for i in range(self.tsp_instance.number_of_cities):
            curr = 1
            for t in range(self.tsp_instance.number_of_cities):
                curr -= self.x(i, t)    
            cost_of_constraint_each_visited += np.power(curr, 2)
            
        cost_of_constraint_each_visited_once = 0
        for t in range(self.tsp_instance.number_of_cities):
            curr = 1
            for i in range(self.tsp_instance.number_of_cities):
                curr -= self.x(i, t)
            cost_of_constraint_each_visited_once += np.power(curr, 2)
        
        cost_of_visiting_cities = 0
        for i, j in itertools.permutations(range(0, self.tsp_instance.number_of_cities), 2):
            curr = 0
            for t in range(self.tsp_instance.number_of_cities):
                inc_t = t + 1
                if inc_t == self.tsp_instance.number_of_cities:
                    inc_t = 0
                curr += self.x(i, t) * self.x(j, inc_t)
            cost_of_visiting_cities += self.tsp_instance.normalized_distance_matrix[i][j] * curr 
        
        cost_operator = A_1 * cost_of_constraint_each_visited + \
                        A_2 * cost_of_constraint_each_visited_once + \
                        B * cost_of_visiting_cities
                
        return cost_operator
    
    def create_mixing_operator(self):
        mixing_operator = 0
        for qubit in self.qubits:
            mixing_operator += cirq.X(qubit)
        return mixing_operator

    def create_qaoa_circuit(self):
        hadamard_circuit_layer = self.create_hadamard_circuit_layer()
        cost_operator = self.create_cost_operator()
        mixing_operator = self.create_mixing_operator()
        parameterized_circuit_layers = tfq.util.exponential(operators = [cost_operator, mixing_operator] * self.p, 
                                                            coefficients = self.parameters)
        qaoa_circuit = hadamard_circuit_layer + parameterized_circuit_layers
        return qaoa_circuit

In [ ]:
tsp_instance = TSP(4)
qaoa_tsp = QAOA_TSP(tsp_instance, p=10, A_1=4, A_2=4, B=1)

Display the circuit (sometimes it takes a little while...).

In [ ]:
# from cirq.contrib.svg import SVGCircuit
# SVGCircuit(qaoa_tsp.circuit)

## Hybrid Quantum-Classical Feed Forward Neural Network

In [ ]:
class QFFNN(tf.keras.layers.Layer):
  def __init__(self, parameters):
    super(QFFNN, self).__init__()
    self.parameters = parameters
    self.params_inp = tf.keras.Input(shape=(len(self.parameters),), name='input_layer')
    self.first_hidden = tf.keras.layers.Dense(len(self.parameters), name="hidden_layer")
    self.expectation = tfq.layers.Expectation(name="expectation_layer")
    

  def call(self, inputs):
    parameterized_circuit = inputs[0]
    cost_operator = inputs[1]
    initial_parameter_values = inputs[2]
    
    parameter_values = self.first_hidden(initial_parameter_values)
    expectation_value = self.expectation(parameterized_circuit,
                                       operators=cost_operator,
                                       symbol_names=self.parameters,
                                       symbol_values=parameter_values)

    return [parameter_values, expectation_value]

In [ ]:
parametrized_circuit_input = tf.keras.Input(shape=(), dtype=tf.dtypes.string)
operator_input = tf.keras.Input(shape=(1,), dtype=tf.dtypes.string)
parameters_input = tf.keras.Input(shape=(2 * qaoa_tsp.p,))

In [ ]:
qffnn = QFFNN(qaoa_tsp.parameters)
output = qffnn([parametrized_circuit_input, operator_input, parameters_input])

In [ ]:
model = tf.keras.Model(
              inputs=[
                  parametrized_circuit_input, 
                  operator_input, 
                  parameters_input
              ],
              outputs=[
                  output[0], # array of optimized 2p parameters
                  output[1], # expectation value
              ])

In [ ]:
model.compile(
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
              loss = tf.keras.losses.mean_absolute_error,
              loss_weights=[0, 1]
             )

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None,)]            0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None, 20)]         0           []                               
                                                                                                  
 qffnn (QFFNN)                  [(None, 20),         420         ['input_1[0][0]',                
                                 (None, 1)]                       'input_2[0][0]',            

In [ ]:
circuit_tensor = tfq.convert_to_tensor([qaoa_tsp.circuit])
cost_operator_tensor = tfq.convert_to_tensor([qaoa_tsp.cost_operator])
initial_parameters = np.zeros((1, qaoa_tsp.p * 2)).astype(np.float32)

In [ ]:
%%time
history = model.fit(
              x=[
                  circuit_tensor, 
                  cost_operator_tensor, 
                  initial_parameters, 
              ],
              y=[
                  np.zeros((1, qaoa_tsp.p * 2)),
                  np.zeros((1, 1)), # the closer to 0 the better the result
              ], 
              epochs=250,
              verbose=1)

Epoch 1/250
1/1 [==============================] - 6s 6s/step - loss: 71.9031 - qffnn_loss: 0.0000e+00 - qffnn_1_loss: 71.9031
Epoch 2/250
1/1 [==============================] - 6s 6s/step - loss: 54.5298 - qffnn_loss: 0.0100 - qffnn_1_loss: 54.5298
Epoch 3/250
1/1 [==============================] - 4s 4s/step - loss: 40.4074 - qffnn_loss: 0.0114 - qffnn_1_loss: 40.4074
Epoch 4/250
1/1 [==============================] - 4s 4s/step - loss: 28.6246 - qffnn_loss: 0.0167 - qffnn_1_loss: 28.6246
Epoch 5/250
1/1 [==============================] - 4s 4s/step - loss: 20.9584 - qffnn_loss: 0.0219 - qffnn_1_loss: 20.9584
Epoch 6/250
1/1 [==============================] - 4s 4s/step - loss: 18.5701 - qffnn_loss: 0.0272 - qffnn_1_loss: 18.5701
Epoch 7/250
1/1 [==============================] - 4s 4s/step - loss: 18.6821 - qffnn_loss: 0.0315 - qffnn_1_loss: 18.6821
Epoch 8/250
1/1 [==============================] - 4s 4s/step - loss: 20.3880 - qffnn_loss: 0.0348 - qffnn_1_loss: 20.3880
Epoch 9/250


In [ ]:
plt.plot(history.history['loss'])
plt.xlabel("Iteration")
plt.ylabel("Loss")
plt.show()

## Results

In [ ]:
parameter_values = model.predict([circuit_tensor, cost_operator_tensor, initial_parameters])[0]
print(f"Parameter values:\n {parameter_values}")

In [ ]:
samples_amount = 2**16
sample_layer = tfq.layers.Sample()
output = sample_layer(circuit_tensor, 
                      symbol_names=qaoa_tsp.parameters, 
                      symbol_values=parameter_values, 
                      repetitions=samples_amount)

In [ ]:
from collections import Counter

results = output.numpy()[0].astype(str).tolist()
results_to_display = [''.join(result) for result in results]
correct_results = ("0001100001000010","0010010010000001","0100100000010010","1000000100100100","1000010000100001","0100001000011000","0001001001001000","0010000110000100","0100000110000010","0010100000010100","0001010000101000","0001100000100100","1000000101000010","1000001001000001","0100001010000001", "0100000100101000", "0010010000011000", "0100100000100001", "1000001000010100", "0001001010000100", "0001010010000010","0010000101001000", "1000010000010010", "0010100001000001")
counts = Counter(results_to_display)

correct_results_count = sum(counts[result] for result in correct_results)
print(f'Correct results: {round(correct_results_count / samples_amount * 100,2)}% \n')

print(f'bin \t\t\t\t occurences \t correct?')
for row in counts.most_common():
    is_correct = row[0] in (correct_results)
    print(f"{row[0][0:4]} \t {row[0][4:8]} \t {row[0][8:12]} \t {row[0][12:]} \t {row[1]} \t\t {is_correct}")